In [1]:
import getopt
import sys
import gc
import os
import random
sys.path.append('/home/atac2rna/program/atac2rna/Model/butterfly/Butterfly')
sys.path.append('/home/atac2rna/program/atac2rna/Model/butterfly/experiment/RNA_ADT')
from data_processing import RNA_data_preprocessing, ATAC_data_preprocessing
import scanpy as sc
import anndata as ad
    
sc_data = sc.read_h5ad('/home/atac2rna/data/atac2rna/data/openproblems_neurips2021/GSE194122_openproblems_neurips2021_multiome_BMMC_count.h5ad')
RNA_data_test = sc_data[:, 0:13431]
ATAC_data_test = sc_data[:, 13431:]
del sc_data
gc.collect()

RNA_data_train = sc.read_h5ad('/home/atac2rna/data/atac2rna/data/openproblems_neurips2021/bmmc_RNA_cnt.h5ad')
ADT_data_train = sc.read_h5ad('/home/atac2rna/data/atac2rna/data/openproblems_neurips2021/bmmc_ADT_CLRed.h5ad')

In [2]:
rna_atac_gene = RNA_data_test.var.index
rna_adt_gene = RNA_data_train.var.index
gene_list_rna_atac = []
gene_list_rna_adt = []
for i in range(len(rna_atac_gene)):
    for j in range(len(rna_adt_gene)):
        if rna_adt_gene[j] == rna_atac_gene[i]:
            gene_list_rna_atac.append(i)
            gene_list_rna_adt.append(j)
            break

In [3]:
import scipy.sparse as sp

In [4]:
big_RNA = ad.AnnData(X = sp.vstack((RNA_data_train.X[:, gene_list_rna_adt], RNA_data_test.X[:, gene_list_rna_atac])))

In [5]:
big_RNA.var = RNA_data_test.var.iloc[gene_list_rna_atac, :]

In [6]:
big_RNA.var

,feature_types,gene_id
AL627309.5,GEX,ENSG00000241860
LINC01409,GEX,ENSG00000237491
LINC01128,GEX,ENSG00000228794
NOC2L,GEX,ENSG00000188976
KLHL17,GEX,ENSG00000187961
...,...,...
MT-ND5,GEX,ENSG00000198786
MT-ND6,GEX,ENSG00000198695
MT-CYB,GEX,ENSG00000198727
AL592183.1,GEX,ENSG00000273748


In [7]:
file_path = '/home/atac2rna/data/atac2rna/model_output/22_10_12/Cite_test/A2R2A/R2A'

In [8]:
############################################################
# Part 1 data processing
big_RNA = RNA_data_preprocessing(
    big_RNA,
    normalize_total=True,
    log1p=True,
    use_hvg=True,
    n_top_genes=3000,
    save_data=False,
    file_path=file_path,
    logging_path=file_path
    )

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [9]:
RNA_data_train = ad.AnnData(X=big_RNA.X[0:90261, :], obs=RNA_data_train.obs, var=big_RNA.var)

In [10]:
RNA_data_test = ad.AnnData(X=big_RNA.X[90261:, :], obs=RNA_data_test.obs, var=big_RNA.var)

In [11]:
id_list = [i for i in range(90261)]

In [12]:
random.seed(19193)
random.shuffle(id_list)
train_id = id_list[0:int(len(id_list) * 0.8)].copy()
validation_id = id_list[int(len(id_list) * 0.8):].copy()

In [13]:
############################################################
# Part 2 split datasets
from split_datasets import *
train_id_r = train_id.copy()
train_id_a = train_id.copy()
validation_id_r = validation_id.copy()
validation_id_a = validation_id.copy()

In [ ]:
from scvi_colab import install
install()
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import scvi
import sys
from calculate_cluster import *
import scipy.sparse as sp
from split_datasets import *


sc_data = sc.read_h5ad('/home/atac2rna/data/atac2rna/data/openproblems_neurips2021/GSE194122_openproblems_neurips2021_cite_BMMC_count.h5ad')
adata_gene = sc_data[:, 0:13953]
adata_protein = sc_data[:, 13953:]

sc.pp.highly_variable_genes(adata_gene, batch_key="Site", flavor="seurat_v3", n_top_genes=4000, subset=True)

sys.path.append('/home/atac2rna/program/atac2rna/Model/22_10_10')

adata_gene_train = adata_gene[train_id].copy()
adata_protein_train = adata_protein[train_id].copy()

adata_gene_train.obsm['protein_expression'] = adata_protein_train.to_df()

scvi.model.TOTALVI.setup_anndata(adata_gene_train, batch_key="Site", protein_expression_obsm_key="protein_expression")

model = scvi.model.TOTALVI(adata_gene_train, latent_distribution="normal", n_layers_decoder=2)

model.train()

adata_gene_train.obsm["X_totalVI"] = model.get_latent_representation()

leiden_adata = ad.AnnData(adata_gene_train.obsm["X_totalVI"])
sc.pp.neighbors(leiden_adata)
sc.tl.leiden(leiden_adata, resolution=3)


In [ ]:
import random
copy_count = 3
random.seed(19193)
cell_type = leiden_adata.obs.leiden
for i in range(len(cell_type.cat.categories)):
    cell_type_name = cell_type.cat.categories[i]
    idx_temp = list(cell_type[cell_type == cell_type_name].index.astype(int))
    for j in range(copy_count - 1):
        random.shuffle(idx_temp)
        for each in idx_temp:
            train_id_r.append(train_id[each])
        random.shuffle(idx_temp)
        for each in idx_temp:
            train_id_a.append(train_id[each])

In [15]:
from train_model_cite import Model
import torch
import torch.nn as nn

RNA_input_dim = 3000
ATAC_input_dim = ADT_data_train.X.shape[1]

R_kl_div = 1 / RNA_input_dim * 20
A_kl_div = R_kl_div
kl_div = R_kl_div + A_kl_div

In [16]:
############################################################
# Part 4 construct model
model = Model(
    R_encoder_nlayer = 2, 
    A_encoder_nlayer = 2,
    R_decoder_nlayer = 2, 
    A_decoder_nlayer = 2,
    R_encoder_dim_list = [RNA_input_dim, 256, 128],
    A_encoder_dim_list = [ATAC_input_dim, 128, 128],
    R_decoder_dim_list = [128, 256, RNA_input_dim],
    A_decoder_dim_list = [128, 128, ATAC_input_dim],
    R_encoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    A_encoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    R_decoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    A_decoder_act_list = [nn.LeakyReLU(), nn.Identity()],
    translator_embed_dim = 128, 
    translator_input_dim_r = 128,
    translator_input_dim_a = 128,
    translator_embed_act_list = [nn.LeakyReLU(), nn.LeakyReLU(), nn.LeakyReLU()],
    discriminator_nlayer = 1,
    discriminator_dim_list_R = [128],
    discriminator_dim_list_A = [128],
    discriminator_act_list = [nn.Sigmoid()],
    dropout_rate = 0.1,
    R_noise_rate = 0.5,
    A_noise_rate = 0,
    chrom_list = [],
    output_path = file_path,
    RNA_data = RNA_data_train,
    ATAC_data = ADT_data_train
)

In [ ]:
############################################################
# Part 5 train model    
model.train(
    R_encoder_lr = 0.001,
    A_encoder_lr = 0.001,
    R_decoder_lr = 0.001,
    A_decoder_lr = 0.001,
    R_translator_lr = 0.001,
    A_translator_lr = 0.001,
    translator_lr = 0.001,
    discriminator_lr = 0.005,
    R2R_pretrain_epoch = 100,
    A2A_pretrain_epoch = 100,
    lock_encoder_and_decoder = False,
    translator_epoch = 200,
    patience = 50,
    batch_size = 64,
    r_loss = nn.MSELoss(size_average=True),
    a_loss = nn.MSELoss(size_average=True),
    d_loss = nn.BCELoss(size_average=True),
    loss_weight = [1, 2, 1, R_kl_div, A_kl_div, kl_div],
    train_id_r = train_id_r,
    train_id_a = train_id_a,
    validation_id_r = validation_id_r, 
    validation_id_a = validation_id_a, 
    save_model = True,
    output_path = file_path,
    seed = 19193,
    kl_mean = True,
    R_pretrain_kl_warmup = 50,
    A_pretrain_kl_warmup = 50,
    translation_kl_warmup = 50,
    load_model = None
)

RNA pretrain:  20%|████▍                 | 20/100 [10:45<47:45, 35.82s/it, train=0.0176, val=0.0184]

In [19]:
model.is_train_finished

True